# 🐕 Random Dog MCP Server & Client Tutorial

*Created by [Brett Sanders](https://brettsanders.com) (with the help of AI)*

This notebook demonstrates how to use the Model Context Protocol (MCP) to create a server that fetches random dog images and how to interact with it using both a custom client and AI agents.

## 📚 What You'll Learn

1. **MCP Basics**: Understanding the Model Context Protocol
2. **Server Architecture**: How the random dog MCP server works
3. **Client Usage**: Using the custom client wrapper
4. **Agent Integration**: Connecting MCP servers with AI agents
5. **Practical Examples**: Running live examples with real dog images

## 📁 Project Files

- `random_dog_server.py`: MCP server that provides dog image tools
- `random_dog_client.py`: Client wrapper for easier interaction
- `random_dog_tutorial.ipynb`: This tutorial notebook


## 🚀 Setup and Imports

First, let's import all the necessary libraries and set up our environment.


In [ ]:
import asyncio
import json
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown, Image
import requests

# Load environment variables
load_dotenv(override=True)

print("✅ Setup complete! Ready to work with random dogs! 🐕")


## 🔧 Understanding the MCP Server

Let's first examine our MCP server structure. The server provides two main tools:

1. **`get_random_dog()`**: Returns complete dog data including URL and file size
2. **`get_dog_image_url()`**: Returns just the image URL

### Server Code Overview

```python
@mcp.tool()
async def get_random_dog() -> Dict[str, Any]:
    """Get a random dog image from the random.dog API."""
    # Fetches from https://random.dog/woof.json
    # Returns: {"url": "...", "fileSizeBytes": 123456, "message": "..."}
```

The server uses the FastMCP framework to provide these tools through the Model Context Protocol, making them accessible to AI agents and other clients.


## 🧪 Testing the MCP Server Directly

Let's start by testing our MCP server directly to see what tools are available.


In [ ]:
# Define MCP server parameters
params = {"command": "uv", "args": ["run", "random_dog_server.py"]}

print("🔍 Testing MCP Server Directly...")
print("=" * 50)

# Test the server
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    # List available tools
    mcp_tools = await server.list_tools()
    print(f"📋 Available tools: {[tool.name for tool in mcp_tools]}")
    print()
    
    # Show tool details
    for tool in mcp_tools:
        print(f"🔧 {tool.name}: {tool.description}")
    
    print("\n" + "=" * 50)
    print("✅ MCP Server is working correctly!")


## 🐕 Getting Your First Random Dog

Now let's fetch our first random dog image using the MCP server tools.


In [ ]:
print("🎲 Fetching a random dog...")

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    # Get complete dog data
    dog_response = await server.call_tool("get_random_dog", {})
    print(f"🐕 Raw MCP response: {dog_response}")
    
    # Get just the URL
    url_response = await server.call_tool("get_dog_image_url", {})
    print(f"🔗 Raw URL response: {url_response}")
    
    # Parse the complete dog data response
    try:
        if dog_response and dog_response.content and len(dog_response.content) > 0:
            json_text = dog_response.content[0].text
            dog_data = json.loads(json_text)
            
            print(f"\n📊 Parsed Dog Data: {dog_data}")
            
            if 'url' in dog_data:
                url = dog_data['url']
                file_size = dog_data.get('fileSizeBytes', 'Unknown')
                
                print(f"\n📊 Dog Image Details:")
                print(f"   URL: {url}")
                print(f"   File Size: {file_size} bytes")
                
                # Store the URL for the next cell
                latest_dog_url = url
                latest_dog_size = file_size
    except Exception as e:
        print(f"❌ Error parsing dog data: {e}")
    
    # Parse the URL-only response
    try:
        if url_response and url_response.content and len(url_response.content) > 0:
            url_text = url_response.content[0].text
            print(f"\n🔗 Parsed URL: {url_text}")
    except Exception as e:
        print(f"❌ Error parsing URL response: {e}")


## 🖼️ Displaying the Dog Image

Let's actually display the dog image in our notebook!


In [ ]:
# Get a fresh dog image for display
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    dog_response = await server.call_tool("get_random_dog", {})

# Parse the MCP response - the actual data is in content[0].text as JSON
try:
    if dog_response and dog_response.content and len(dog_response.content) > 0:
        # Extract the JSON string from the response
        json_text = dog_response.content[0].text
        dog_data = json.loads(json_text)
        
        if 'url' in dog_data and dog_data['url']:
            url = dog_data['url']
            file_size = dog_data.get('fileSizeBytes', 'Unknown')
            
            print(f"🐕 Here's your random dog!")
            print(f"📊 File size: {file_size} bytes")
            print(f"🔗 URL: {url}")
            print()
            
            try:
                # Display the image
                display(Image(url=url, width=400))
            except Exception as e:
                print(f"❌ Could not display image: {e}")
                print(f"But you can view it at: {url}")
        else:
            print(f"❌ No valid URL in response: {dog_data}")
    else:
        print(f"❌ Invalid response format: {dog_response}")
except json.JSONDecodeError as e:
    print(f"❌ Error parsing JSON response: {e}")
    print(f"Raw response: {dog_response}")
except Exception as e:
    print(f"❌ Error processing response: {e}")
    print(f"Response: {dog_response}")


## 🎯 Using the Custom Client

Now let's use our custom `RandomDogMCPClient` which provides a more convenient interface.


In [ ]:
# Import our custom client
from random_dog_client import RandomDogMCPClient

print("🎯 Testing Custom RandomDogMCPClient...")
print("=" * 50)

async with RandomDogMCPClient() as client:
    # List available tools
    tools = await client.list_tools()
    print(f"📋 Available tools: {[tool.name for tool in tools]}")
    
    # Get random dog data using the client
    print("\n🐕 Getting dog data via client...")
    dog_response = await client.get_random_dog()
    
    # Parse the client response (it also returns MCP format)
    try:
        if dog_response and dog_response.content and len(dog_response.content) > 0:
            json_text = dog_response.content[0].text
            dog_info = json.loads(json_text)
            print(f"Dog info: {dog_info}")
    except Exception as e:
        print(f"Dog response: {dog_response}")
    
    # Get just the URL using the client
    print("\n🔗 Getting dog URL via client...")
    url_response = await client.get_dog_image_url()
    try:
        if url_response and url_response.content and len(url_response.content) > 0:
            dog_url = url_response.content[0].text
            print(f"Dog URL: {dog_url}")
    except Exception as e:
        print(f"URL response: {url_response}")
    
    print("\n✅ Custom client working perfectly!")
    print("🎯 The client provides a convenient wrapper around the MCP server calls!")


## 🤖 AI Agent Integration

Now for the exciting part - let's create an AI agent that can use our dog tools!


In [ ]:
# Create an AI agent that can fetch dog images
instructions = """You are a helpful and enthusiastic dog-loving assistant! 🐕

You have access to tools that can fetch random dog images from the internet. 
When someone asks for a dog image, use your tools to get one and provide:
1. The image URL
2. The file size information
3. An enthusiastic comment about dogs

Always be excited and positive about dogs!"""

request = "Hi! I'm having a bad day. Can you cheer me up with a cute random dog image?"
model = "gpt-4o-mini"

print("🤖 Creating Dog-Loving AI Agent...")
print("=" * 50)

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    # Create the agent with our MCP server
    agent = Agent(
        name="dog_enthusiast", 
        instructions=instructions, 
        model=model, 
        mcp_servers=[mcp_server]
    )
    
    # Run the agent with tracing
    with trace("dog_agent_demo"):
        result = await Runner.run(agent, request)
        
    print("🐕 Agent Response:")
    print("=" * 30)
    display(Markdown(result.final_output))


## 📋 Summary and Next Steps

### What We've Learned

1. **MCP Server Setup**: How to create tools that fetch external data
2. **Client Wrapping**: Building convenient client interfaces  
3. **Agent Integration**: Connecting MCP servers with AI agents
4. **Error Handling**: Robust error handling for external APIs
5. **Practical Applications**: Real-world usage patterns

### Key Components

- **`random_dog_server.py`**: FastMCP server with dog image tools
- **`random_dog_client.py`**: Convenient client wrapper
- **Agent Integration**: Seamless AI agent connectivity

### Next Steps

1. **Extend the Server**: Add more tools (cat images, specific breeds, etc.)
2. **Add Caching**: Implement local caching for better performance
3. **Error Recovery**: Add retry logic and fallback mechanisms
4. **Multi-API Support**: Connect to multiple pet image APIs
5. **Advanced Agents**: Create specialized agents for different use cases

### 🚀 Ready to Build More?

You now have a solid foundation for:
- Creating your own MCP servers
- Building client wrappers
- Integrating with AI agents
- Handling external APIs safely

Happy coding! 🐕🎉


## 🔧 Utility Functions

Here are some utility functions you can use in your own projects:


In [ ]:
# Utility function for simple dog fetching using direct MCP server
async def get_simple_dog():
    """Simple utility to get a random dog with error handling"""
    try:
        async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
            dog_response = await server.call_tool("get_random_dog", {})
            if dog_response and dog_response.content and len(dog_response.content) > 0:
                json_text = dog_response.content[0].text
                return json.loads(json_text)
            else:
                return {"error": "Invalid response format", "url": "", "fileSizeBytes": 0}
    except Exception as e:
        return {"error": str(e), "url": "", "fileSizeBytes": 0}

# Test the utility
print("🔧 Testing utility function...")
simple_dog = await get_simple_dog()
print(f"Simple dog result: {simple_dog}")

# Another utility for batch fetching
async def get_multiple_dogs(count=3):
    """Get multiple random dogs using direct MCP server"""
    dogs = []
    try:
        async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
            for i in range(count):
                try:
                    dog_response = await server.call_tool("get_random_dog", {})
                    if dog_response and dog_response.content and len(dog_response.content) > 0:
                        json_text = dog_response.content[0].text
                        dog_data = json.loads(json_text)
                        dogs.append(dog_data)
                    else:
                        dogs.append({"error": "Invalid response"})
                except Exception as e:
                    dogs.append({"error": str(e)})
    except Exception as e:
        return [{"error": f"Server connection failed: {str(e)}"}]
    return dogs

# Test batch fetching
print("\n📦 Testing batch utility...")
batch_dogs = await get_multiple_dogs(2)
print(f"Batch result: Got {len(batch_dogs)} dogs")
for i, dog in enumerate(batch_dogs, 1):
    if 'url' in dog and dog['url']:
        print(f"   Dog {i}: {dog['url'][:50]}...")
    else:
        print(f"   Dog {i}: Error - {dog.get('error', 'Unknown error')}")


## 🔍 Troubleshooting

### Common Issues

1. **Server Won't Start**
   - Make sure `uv` is installed: `pip install uv`
   - Check that all dependencies are available
   - Verify the server file path is correct

2. **No Images Displaying**
   - Check your internet connection
   - The random.dog API might be temporarily unavailable
   - Some images might be in formats that don't display well in notebooks

3. **Agent Not Using Tools**
   - Verify the MCP server is properly connected
   - Check that tools are listed correctly
   - Make sure your OpenAI API key is set

4. **Import Errors**
   - Ensure you're in the correct directory
   - Check that all required packages are installed
   - Verify Python path includes the current directory

### Debug Commands

```python
# Test basic connectivity
import requests
response = requests.get("https://random.dog/woof.json")
print(response.status_code, response.json())

# Test MCP server manually
# Run in terminal: uv run random_dog_server.py
```

### File Structure

Make sure your files are organized like this:

```
random_dog_mcp_server_client/
├── random_dog_server.py
├── random_dog_client.py
└── random_dog_tutorial.ipynb
```
